# Convert Keras to TF-Lite

In [2]:
import tensorflow as tf
from tensorflow import keras

In [6]:
model = keras.models.load_model('xception_v4_48_0.886.h5')

2023-12-27 21:28:09.119103: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-27 21:28:09.126148: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-27 21:28:09.126745: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-27 21:28:09.127667: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [7]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('xception_v4_48_0.886.tflite', 'wb') as f_out:
    f_out.write(tflite_model)


INFO:tensorflow:Assets written to: /tmp/tmpbhlnn8nb/assets


INFO:tensorflow:Assets written to: /tmp/tmpbhlnn8nb/assets
2023-12-27 21:30:11.769173: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-12-27 21:30:11.769218: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-12-27 21:30:11.769962: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpbhlnn8nb
2023-12-27 21:30:11.798440: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2023-12-27 21:30:11.798472: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /tmp/tmpbhlnn8nb
2023-12-27 21:30:11.889284: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2023-12-27 21:30:11.916307: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-12-27 21:30:12.494414: I tensorflow/cc/saved_model/loader.cc:212] Running initializatio

In [8]:
!pip install keras-image-helper

In [9]:
!pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

Looking in indexes: https://pypi.org/simple, https://google-coral.github.io/py-repo/


In [1]:
#import tensorflow.lite as tflite
import tflite_runtime.interpreter as tflite
from keras_image_helper import create_preprocessor

In [2]:
interpreter = tflite.Interpreter(model_path='xception_v4_48_0.886.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [4]:
path = "./evaluation/Fried food/107.jpg"
X = preprocessor.from_path(path)

In [5]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [6]:
classes = [
    'Bread',
    'Dairy product',
    'Dessert',
    'Egg',
    'Fried food',
    'Meat',
    'Noodles-Pasta',
    'Rice',
    'Seafood',
    'Soup',
    'Vegetable-Fruit'
]

dict(zip(classes, preds[0]))

{'Bread': -1.1156602,
 'Dairy product': -19.08321,
 'Dessert': -15.672163,
 'Egg': -15.45562,
 'Fried food': 24.231289,
 'Meat': -1.8954746,
 'Noodles-Pasta': -24.310883,
 'Rice': -27.576273,
 'Seafood': -23.818464,
 'Soup': -21.958662,
 'Vegetable-Fruit': -8.89721}

In [11]:
prediction_result = dict(zip(classes, preds[0]))

max_class = max(prediction_result, key=prediction_result.get)
print(f"The predicted class: {max_class}")

The predicted class: Fried food
